# Setup

In [1]:
from pathlib import Path
import datetime
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf
from tensorflow import keras

print('TF version:', tf.__version__)

print('Available devices:')
for device in tf.config.list_physical_devices():
    print(f'  {device.device_type}, {device.name}')

gpus = tf.config.list_physical_devices('GPU')
cpus = tf.config.list_physical_devices('CPU')

if gpus:
    # limit GPU visibility to last GPU
    tf.config.set_visible_devices([cpus[0], gpus[-1]])
    # allow GPU memory growth
    tf.config.experimental.set_memory_growth(gpus[-1], True)

print('Visible devices:')
for device in tf.config.get_visible_devices():
    print(f'  {device.device_type}, {device.name}')

%load_ext autoreload
%autoreload 1

%aimport model

TF version: 2.4.1
Available devices:
  CPU, /physical_device:CPU:0
  GPU, /physical_device:GPU:0
  GPU, /physical_device:GPU:1
Visible devices:
  CPU, /physical_device:CPU:0
  GPU, /physical_device:GPU:1


# Read data

In [2]:
# read signals and labels

datafile = Path('data/labeled-elm-events-smithdr.hdf5')

super_window_size = 250

signal_window_size = 8  # size of signal window in time domain for model input; e.g. 4, 8, 16
label_look_ahead = 0  # label look ahead from end of signal window; e.g. 0, 1, 2
total_window_size = signal_window_size + label_look_ahead

max_elms = None

with h5py.File(datafile, 'r') as hf:
    print(f'Datafile: {datafile.as_posix()}')
    print(f'Number of ELM events: {len(hf)}')
    # loop over ELM events
    for ielm, elm_event in enumerate(hf.values()):
        if max_elms and ielm >= max_elms:
            print(f'Limiting data read to {max_elms} ELM events')
            break
        n_super_windows = elm_event['labels'].size // super_window_size
        # load BES signals and reshape
        elm_signals_np = elm_event['signals'][..., 0:n_super_windows*super_window_size]
        elm_signals_np = elm_signals_np.T.reshape(n_super_windows, super_window_size, 8, 8)
        # load ELM labels and reshape
        elm_labels_np = elm_event['labels'][0:n_super_windows*super_window_size]
        elm_labels_np = elm_labels_np.reshape(n_super_windows, super_window_size)
        # construct valid_t0 mask
        valid_t0_np = np.ones(elm_labels_np.shape, dtype=np.int8)
        valid_t0_np[:,-(total_window_size-1):] = 0
        # convert to tensors
        signals_tmp = tf.convert_to_tensor(elm_signals_np, dtype=tf.float32)
        labels_tmp = tf.convert_to_tensor(elm_labels_np)
        valid_t0_tmp = tf.convert_to_tensor(valid_t0_np)
        if ielm == 0:
            # initialize tensors
            signals = signals_tmp
            labels = labels_tmp
            valid_t0 = valid_t0_tmp
        else:
            # concat new data
            signals = tf.concat([signals, signals_tmp], 0)
            labels = tf.concat([labels, labels_tmp], 0)
            valid_t0 = tf.concat([valid_t0, valid_t0_tmp], 0)
            
# normalize signals
signals = signals / np.max(np.abs(signals))
            
n_times = np.prod(labels.shape)
n_elm_times = np.count_nonzero(np.array(labels) == 1)
print(f'Total time points: {n_times}')
print(f'Fraction active ELM: {n_elm_times/n_times*100.0:.2f} %')
print(f'Fraction no ELM: {(n_times-n_elm_times)/n_times*100.0:.2f} %')

for tensor in [signals, labels, valid_t0]:
    print(f'shape {tensor.shape} dtype {tensor.dtype} device {tensor.device[-5:]} min {np.min(tensor)} max {np.max(tensor)}')

Datafile: data/labeled-elm-events-smithdr.hdf5
Number of ELM events: 52
Total time points: 311250
Fraction active ELM: 5.15 %
Fraction no ELM: 94.85 %
shape (1245, 250, 8, 8) dtype <dtype: 'float32'> device GPU:0 min -0.9876285791397095 max 1.0
shape (1245, 250) dtype <dtype: 'int8'> device GPU:0 min 0 max 1
shape (1245, 250) dtype <dtype: 'int8'> device GPU:0 min 0 max 1


# Prepare data

In [3]:
# shuffle super-windows

n_super_windows = signals.shape[0]
shuffled_indices = tf.random.shuffle(tf.range(n_super_windows))

def apply_shuffle(tensor):
    return tf.gather(tensor, shuffled_indices, axis=0)

print('Shuffling super windows ...')

signals = apply_shuffle(signals)
labels = apply_shuffle(labels)
valid_t0 = apply_shuffle(valid_t0)

print('Tensors with shuffled super windows')
for tensor in [signals, labels, valid_t0]:
    print(f'shape {tensor.shape} dtype {tensor.dtype} device {tensor.device[-5:]}')

Shuffling super windows ...
Tensors with shuffled super windows
shape (1245, 250, 8, 8) dtype <dtype: 'float32'> device GPU:0
shape (1245, 250) dtype <dtype: 'int8'> device CPU:0
shape (1245, 250) dtype <dtype: 'int8'> device CPU:0


In [4]:
# reshape and make valid indices

# temp reshapes
signals_temp = tf.reshape(signals, [-1,8,8])
labels_temp = tf.reshape(labels, [-1])
valid_t0_temp = tf.reshape(valid_t0, [-1])

# make valid indices
valid_indices_temp = np.arange(valid_t0_temp.shape[0])
valid_indices_temp[valid_t0_temp == 0] = -1  # final usage for `valid_t0` tensors; safe to delete
valid_indices_temp = tf.convert_to_tensor(valid_indices_temp, dtype=tf.int32)

print('Reshaped tensors with concatenated super windows')
for tensor in [signals_temp, labels_temp, valid_indices_temp]:
    print(f'shape {tensor.shape} dtype {tensor.dtype} device {tensor.device[-5:]}')

Reshaped tensors with concatenated super windows
shape (311250, 8, 8) dtype <dtype: 'float32'> device GPU:0
shape (311250,) dtype <dtype: 'int8'> device GPU:0
shape (311250,) dtype <dtype: 'int32'> device CPU:0


In [5]:
# partition into training, validation, and testing

fraction_validate = 0.05  # validation data for evaluation after each epoch
fraction_test = 0.25  # test data for evaluation after training

n_validate = np.int(fraction_validate * n_super_windows)
n_test = np.int(fraction_test * n_super_windows)
n_train = n_super_windows - n_test - n_validate
print(f'Super window partition: n_train {n_train} n_validate {n_validate} n_test {n_test}')

def partition_data(tensor):
    cut_1 = n_train * super_window_size
    cut_2 = cut_1 + (n_validate * super_window_size)
    return (tensor[:cut_1, ...],  # training partition
        tensor[cut_1:cut_2, ...],  # validation partition
        tensor[cut_2:, ...],  # testing partition
        )

# partition
signals_train, signals_validate, signals_test = partition_data(signals_temp)
labels_train, labels_validate, labels_test = partition_data(labels_temp)
valid_indices_train, valid_indices_validate, valid_indices_test = partition_data(valid_indices_temp)

del(signals_temp, labels_temp, valid_t0_temp, valid_indices_temp)

print('Training tensors')
for tensor in [signals_train, labels_train, valid_indices_train]:
    print(f'shape {tensor.shape} dtype {tensor.dtype} device {tensor.device[-5:]}')

Super window partition: n_train 872 n_validate 62 n_test 311
Training tensors
shape (218000, 8, 8) dtype <dtype: 'float32'> device GPU:0
shape (218000,) dtype <dtype: 'int8'> device GPU:0
shape (218000,) dtype <dtype: 'int32'> device GPU:0


In [6]:
# remove invalid `-1` elements from valid indices tensors

# resulting tensors contain valid indices for indexing into `signals` and `labels`
# to generate signal windows and labels.

def remove_invalid_incides_and_shuffle(tensor):
    tensor = tensor[tensor != -1]
    tesnor = tf.random.shuffle(tensor)
    assert(np.all(tensor >=0))
    return tensor

valid_indices_train = remove_invalid_incides_and_shuffle(valid_indices_train)
valid_indices_validate = remove_invalid_incides_and_shuffle(valid_indices_validate)
valid_indices_test = remove_invalid_incides_and_shuffle(valid_indices_test)

print('Training tensors')
for tensor in [signals_train, labels_train, valid_indices_train]:
    print(f'shape {tensor.shape} dtype {tensor.dtype} device {tensor.device[-5:]}')

Training tensors
shape (218000, 8, 8) dtype <dtype: 'float32'> device GPU:0
shape (218000,) dtype <dtype: 'int8'> device GPU:0
shape (211896,) dtype <dtype: 'int32'> device GPU:0


In [7]:
# create datasets with valid indices

def make_generator(valid_indices, signals, labels):
    def generator():
        i = 0
        while i < valid_indices.shape[0]:
            signal_window = tf.reshape(signals[i:i+signal_window_size, ...], 
                                       [signal_window_size,8,8,1])
            label = tf.reshape(labels[i+signal_window_size+label_look_ahead], [1,])
            i += 1
            yield signal_window, label
    return generator

generator_train = make_generator(valid_indices_train, signals_train, labels_train)
generator_validate = make_generator(valid_indices_validate, signals_validate, labels_validate)
generator_test = make_generator(valid_indices_test, signals_test, labels_test)

dtypes = (signals_train.dtype, labels_train.dtype)
shapes = (tf.TensorShape([signal_window_size,8,8,1]), tf.TensorShape([1]))

# create datasets
ds_train = tf.data.Dataset.from_generator(generator_train, dtypes, output_shapes=shapes)
ds_validate = tf.data.Dataset.from_generator(generator_validate, dtypes, output_shapes=shapes)
ds_test = tf.data.Dataset.from_generator(generator_test, dtypes, output_shapes=shapes)

ds_train

<FlatMapDataset shapes: ((8, 8, 8, 1), (1,)), types: (tf.float32, tf.int8)>

In [8]:
# batch, prefetch, and cache

training_batch_size = 4

# def batch_and_prefetch(dataset):
#     return dataset.batch(batch_size).prefetch(16).shuffle(1000, reshuffle_each_iteration=True)

# ds_train = batch_and_prefetch(ds_train)
# ds_validate = batch_and_prefetch(ds_validate)
# ds_test = batch_and_prefetch(ds_test)

ds_train = ds_train.batch(training_batch_size).shuffle(2000, reshuffle_each_iteration=True).prefetch(tf.data.AUTOTUNE)
ds_validate = ds_validate.batch(16).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(16).prefetch(tf.data.AUTOTUNE)

ds_train

<PrefetchDataset shapes: ((None, 8, 8, 8, 1), (None, 1)), types: (tf.float32, tf.int8)>

# Train model

In [9]:
# initiate model, compile, and verify single evaluation

test_model = model.cnn_model(
    n_lookback=signal_window_size,
    n_filters_1=14,
    n_filters_2=18,
    n_dense_1=50,
    n_dense_2=30,
    dropout_rate=0.1,
    l2_factor=1e-3,
)

batches_per_epoch = valid_indices_train.shape[0] // training_batch_size

lr_schedule = model.CustomSchedule(
    initial_learning_rate=1e-3,
    batches_per_epoch=batches_per_epoch,
    epochs_per_halving=2,
)

optimizer = keras.optimizers.SGD(
    learning_rate=lr_schedule,
)

loss = keras.losses.BinaryCrossentropy()

metrics = [
    keras.metrics.BinaryCrossentropy(),
    keras.metrics.BinaryAccuracy(),
]

log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print(f'log dir: {log_dir}')

# file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
# file_writer.set_as_default()


tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, 
    histogram_freq=1,
    update_freq=5000,
)

earlystop_callback = tf.keras.callbacks.EarlyStopping(
    min_delta=5e-3,
    patience=6,
    verbose=1
)

test_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics,
    weighted_metrics=metrics,
)

sample_output = test_model.evaluate(
    x=ds_train,
    steps=1,
    verbose=0)

print(sample_output)

Filter 1 shape (8, 3, 3) count 14 params 1022
Filter 2 shape (1, 3, 3) count 18 params 2286
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 8, 8, 1)]      0         
_________________________________________________________________
conv3d (Conv3D)              (None, 1, 6, 6, 14)       1022      
_________________________________________________________________
dropout (Dropout)            (None, 1, 6, 6, 14)       0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 1, 4, 4, 18)       2286      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 4, 4, 18)       0         
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
___________________________________

In [10]:
history = test_model.fit(
    x=ds_train,
    epochs=20,
    validation_data=ds_validate,
    workers=2,
    use_multiprocessing=True,
    class_weight={0: 0.05,
                  1: 0.95},
    callbacks=[tensorboard_callback,
               earlystop_callback,
               ],
    )

Epoch 1/20
52974/52974 [==============================] - 308s 6ms/step - loss: 0.2623 - weighted_binary_crossentropy: 0.4500 - weighted_binary_accuracy: 0.9379 - val_loss: 0.3690 - val_weighted_binary_crossentropy: 0.2495 - val_weighted_binary_accuracy: 0.9748
Epoch 2/20
52974/52974 [==============================] - 309s 6ms/step - loss: 0.2267 - weighted_binary_crossentropy: 0.3940 - weighted_binary_accuracy: 0.9554 - val_loss: 0.3438 - val_weighted_binary_crossentropy: 0.2382 - val_weighted_binary_accuracy: 0.9731
Epoch 3/20
52974/52974 [==============================] - 307s 6ms/step - loss: 0.2151 - weighted_binary_crossentropy: 0.3845 - weighted_binary_accuracy: 0.9558 - val_loss: 0.3276 - val_weighted_binary_crossentropy: 0.2308 - val_weighted_binary_accuracy: 0.9730
Epoch 4/20
52974/52974 [==============================] - 300s 6ms/step - loss: 0.2070 - weighted_binary_crossentropy: 0.3735 - weighted_binary_accuracy: 0.9568 - val_loss: 0.3142 - val_weighted_binary_crossentropy